## Library and utils function

In [5]:
import numpy as np
import random
import os
import json
from numpyencoder import NumpyEncoder


In [6]:
# for i in range(1, 6):
#     os.mkdir("Type" + str(i) + "Small")
#     os.mkdir("Type" + str(i) + "Large")


In [7]:
def createRandomSortedList(low=1, high=100, size=1):
    arr = []
    tmp = np.random.randint(low, high)
    for x in range(int(size)):
        while tmp in arr:
            tmp = random.randint(low, high)
        arr.append(tmp)

    arr.sort()
    return np.array(arr)


# Create empty data function

In [8]:
def create_data(N, m, M, r, K1, K2, a):
    """
    Params:
        N: number of fields
        m: lower bound of number of field to harvest
        M: upper bound of number of field to harvest 
        r: max total time between start time and end time to harvest
        K1: lower bound of number of field to harvest in a day
        K2: upper bound of number of field to harvest in a day
        a : number of days that is not harvested
    Return:
        d: list of harvested quantity
        s: list of start time to harvest
        e: list of end time to harvest
    """
    # estimated limit days
    if K1 == K2:
        rangeDay = int(N/K1) + 1
    else:
        rangeDay = np.random.randint(N/K2, N/K1) + 1

    # yield per day
    amount = np.random.randint(m, M + 1, size=rangeDay-a)
    amount = np.sort(amount)

    # the number of fields of that day was chosen at random
    count = np.array([int(N/(rangeDay-a))]*(rangeDay-a))
    while (np.sum(count) < N):
        for i in range(rangeDay-a):
            # prioritize dividing large fields first
            day = (rangeDay-a) - 1 - i
            count[day] += 1
            if (np.sum(count) == N):
                break

    amount_final = []
    count_final = []
    amount = np.append(amount, [0] * a)
    count = np.append(count, [0] * a)
    randomOrder = np.arange(rangeDay)
    np.random.shuffle(randomOrder)
    for random_i in randomOrder:
        amount_final.append(amount[random_i])
        count_final.append(count[random_i])

    # divide the yield among the fields of the same day
    d = []
    s = []
    e = []
    for day in range(rangeDay):
        if amount[day] > 0:
            mark = createRandomSortedList(1, amount[day], size=count[day] + 1)
            mark[0] = 0
            mark[int(count[day])] = amount[day]
            for j in range(int(count[day])):
                d.append(mark[j + 1] - mark[j])
                start = max(day - np.random.randint(r), 0)
                end = min(day + np.random.randint(r), rangeDay - 1)
                s.append(start)
                e.append(end)
    e = np.array(e) - min(s)
    s = np.array(s) - min(s)

    d_final = []
    s_final = []
    e_final = []
    randomOrder = np.arange(N)
    np.random.shuffle(randomOrder)
    for random_i in randomOrder:
        d_final.append(d[random_i])
        s_final.append(s[random_i])
        e_final.append(e[random_i])
    return d_final, s_final, e_final


# Create data

In [12]:
r = [5, 5, 5, 10, 10]
K1 = [4, 2, 10, 4, 2]
K2 = [6, 3, 10, 6, 3]
low = [10, 20, 30, 40, 50]
high = [15, 30, 50, 75, 100]
N = [10, 20, 30, 40, 50]
for k in range(5):
    for i in range(5):
        for j in range(5):
            alter = N[i] // K2[k] - N[i] // low[j]
            if alter > 0:
                a = np.random.randint(0, alter + 1)
            else:
                a = 0
            with open(f"Type{k+1}Small/sample_{N[i]}_{low[j]}_{high[j]}.json", "w") as f:
                d, s, e = create_data(
                    N[i], low[j], high[j], r[k], K1[k], K2[k], a)
                json.dump({"N": N[i], "m": low[j], "M": high[j],
                          "d": d, "s": s, "e": e}, f, cls=NumpyEncoder)


In [13]:
r = [5, 5, 5, 10, 10]
K1 = [4, 2, 10, 4, 2]
K2 = [6, 3, 10, 6, 3]
low = [20, 30, 40, 50, 60]
high = [30, 50, 75, 100, 125]
N = [100, 250, 500, 750, 1000]
for k in range(5):
    for i in range(5):
        for j in range(5):
            alter = N[i] // K2[k] - N[i] // low[j]
            if alter > 0:
                a = np.random.randint(0, alter + 1)
            else:
                a = 0
            with open(f"Type{k+1}Large/sample_{N[i]}_{low[j]}_{high[j]}.json", "w") as f:
                d, s, e = create_data(
                    N[i], low[j], high[j], r[k], K1[k], K2[k], a)
                json.dump({"N": N[i], "m": low[j], "M": high[j],
                          "d": d, "s": s, "e": e}, f, cls=NumpyEncoder)
